In [40]:
import requests
import pandas as pd
import time

In [48]:
# the components of the API call, which make up the "request" variable
url = 'https://api.legiscan.com/?key='
key = 'a7f690aee8bd0c687209f3370df1d86d'
page = 1
request = requests.get(url + key + '&op=getSearch&state=ALL&query=controlled substance' + '&page=' + str(page))

# to print out the full URL, and we can paste in the browser to get
# an interactive look at the raw results
print(url + key + '&op=getSearch&state=ALL&query=controlled substance' + '&page=' + str(page))

https://api.legiscan.com/?key=a7f690aee8bd0c687209f3370df1d86d&op=getSearch&state=ALL&query=controlled substance&page=1


In [49]:
# get the page_total and the count from the request summary
page_total = request.json()['searchresult']['summary']['page_total']
count = request.json()['searchresult']['summary']['count']
print('Page total: ' + str(page_total) + '\n' + 'Total results: ' + str(count))

Page total: 60
Total results: 2973


In [50]:
# request the additional pages of the query by adding 1 to the 'page'
# parameter until it reaches the page_total. Store each page of
# requests in a list. Wait 3 seconds between each request to avoid
# overloading the API

pages = []
for i in range(page_total):
    page = i + 1
    request = requests.get(url + key + '&op=getSearch&state=ALL&query=controlled substance' + '&page=' + str(page))
    time.sleep(3)
    pages.append(request.json())

KeyboardInterrupt: 

In [44]:
# for each page of the request, parse the results and add them to a
# dataframe. each page is a json file with individual results labeled
# '0' through '49' and # 'summary' nested under searchresult. Ignore
# the summary and use pandas.concat to add each of the results from
# the request in a dataframe

df = pd.DataFrame()
for page in pages:
    results = page['searchresult']
    # if the page has no results, skip it
    for i in range(50):
        if str(i) in results:
            df = pd.concat([df, pd.DataFrame(results[str(i)], index=[i])])
        else:
            continue

In [45]:
print(df)

    relevance state bill_number  bill_id                       change_hash  \
0         100    MA       H4255  1834921  acb19d4a4228fc0fa479d5ef78d09027   
1          99    VT       S0114  1723792  6528a67099031816f57c8c46feeaf884   
2          99    AK       HB228  1799829  ddbc084f10b6494e6955c01fa4b5f2ca   
3          99    AK       SB166  1799764  daaa7068ad57eb97c54d28ba2882c4c1   
4          99    MA       H4607  1876101  08167ba9208a1fe52e1bb145652db1d2   
5          99    VT       H0371  1719698  7052f9e3e83a2cc5deb34d97ee4976ab   
6          99    MN      SF1954  1713930  0f689e6b5c7e2d1465e63bf9232f5a27   
7          99    MN      HF1884  1710981  7291f1660c2f9d3d2bc9255ca9398da0   
8          98    US      HB3684  1768824  69dfedb064789a75e134b15e48417cfc   
9          98    US      HB7347  1847774  c252f25983934f4ad167ba35343bb426   
10         98    NC        H727  1756580  af306b4a2645ff0ecfc41a4c44d3f9d9   
11         96    NY      A08349  1791818  36f7afbe33c5f9fc5bf1d2

In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 49 entries, 0 to 48
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   relevance         49 non-null     int64 
 1   state             49 non-null     object
 2   bill_number       49 non-null     object
 3   bill_id           49 non-null     int64 
 4   change_hash       49 non-null     object
 5   url               49 non-null     object
 6   text_url          49 non-null     object
 7   research_url      49 non-null     object
 8   last_action_date  49 non-null     object
 9   last_action       49 non-null     object
 10  title             49 non-null     object
dtypes: int64(2), object(9)
memory usage: 4.6+ KB


In [47]:
df.to_csv('legiscan_psychedelic_results.csv')